In [1]:
# 라이브러리 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip3 install konlpy
!pip install chardet


# JAVA_HOME 환경변수 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64" 

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,776 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,354 kB]
Get:14 http://ar

In [2]:
##사전 준비
import os
import sys
import urllib.request
import pandas as pd
import json
import re
import chardet
from konlpy.tag import Kkma, Komoran, Hannanum, Okt


kkma = Kkma()
okt=Okt()
story=["스토리", "이야기", "내용", "줄거리", "개연성", "결말", "서사", "대사"]
expression=["연출", "OST", "ost", "음악", "영상미", "효과음", "CG", "cg","Cg","편집", "트랙", "작화", "씬", "액션"]

#띄어쓰기
def space(sent):
  sents_pos = okt.pos(sent)
  new_sent = ""
  for word_pos in sents_pos:
      if word_pos[1] in ['Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation']:
          new_sent = new_sent + word_pos[0]
      else:
          new_sent = new_sent + " "+word_pos[0]
  return new_sent

#불용어 제거
def clean_korean_documents(sents):
    for i, document in enumerate(sents):
        clean_words = []
        for word in okt.pos(document, stem=True): 
            if word[1] in ['Noun', 'Verb', 'Adjective']: 
                clean_words.append(word[0])
        sent = ' '.join(clean_words)
        sents[i] = sent

    f = open('stopword_ko.txt', 'r', encoding='utf8')
    stopwords_ko = f.readlines()
    
    for i, sent in enumerate(sents):
        clean_words = []
        doc=sent.split(" ") 
        for word in doc: 
            if word not in stopwords_ko: 
                clean_words.append(word)
        sent = ' '.join(clean_words)
        sents[i] = sent

    return sents

In [3]:
#기계학습
data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t', keep_default_na=False)

reviews = data['document'].to_list()
label = data['label'].to_list()

for i in range(len(reviews)):
  reviews[i]=okt.normalize(reviews[i])
  reviews[i]=space(reviews[i])
reviews= clean_korean_documents(reviews)



In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfd=TfidfVectorizer()
Reviews=tfd.fit_transform(reviews)

lr=LogisticRegression()
lr.fit(Reviews, label)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [5]:
##데이터 가져오기
client_id = "erhyEarWYdIa3BK7YZSL"
client_secret = "XiwsboFmAS"

movie_title=input("영화제목을 입력하세요: ")
query = urllib.parse.quote(movie_title+" 리뷰")
display=100

url = "https://openapi.naver.com/v1/search/blog?query=" + query + '&display=' + str(display) 

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200): 
    response_body = response.read()
    
else:
    print("Error Code:" + rescode)

body=response_body.decode("utf-8")
body=body.replace('"', '')

List=body.split("\n\t\t{\n\t\t\t")
LIST=[]
for i in List:
  if 'naver' in i:
    LIST.append(i)

#링크 추출
links=[]
for i in LIST:
  letter=i.split("\n\t\t\t")
  for j in letter:
    if "link:" in j and "blogger" not in j:
      j.strip()
      j=j.replace("\\", "")
      links.append(j[5:])

#본문 내용 가져오기
from bs4 import BeautifulSoup

true_links=[]
for link in links:
  blog_url=link
  blog=urllib.request.urlopen(blog_url)
  soup=BeautifulSoup(blog.read(), "html.parser")
  true_link="http://blog.naver.com/"+soup.iframe["src"]
  true_links.append(true_link)

contents=[]
for true_link in true_links:
  Blog=urllib.request.urlopen(true_link)
  Soup=BeautifulSoup(Blog.read(), "html.parser")
  if Soup.find("div", {"class":"se-main-container"}) is not None:
    content=Soup.find("div", {"class":"se-main-container"}).text
  else:
    content=Soup.find("div", {"id":"content-area"}).text
  contents.append(content)



영화제목을 입력하세요: 탑건: 매버릭


In [6]:
##내용물 감정분석
story_scores=[]
expression_scores=[]

for i in range(len(contents)):
  encode=chardet.detect(contents[i].encode())
  if encode["encoding"]==None:
    continue
  
  contents[i]=contents[i].strip()
  contents[i]=contents[i].replace(" ", "")
  contents[i]=contents[i].replace("\n", "")
  contents[i]=contents[i].replace("\u200b", "")
  contents[i]=okt.normalize(contents[i])
  sents=kkma.sentences(contents[i])
  for j in range(len(sents)):
    sents[j]=space(sents[j])
           
  sents=clean_korean_documents(sents) 
  
  story_sents=[]
  expression_sents=[]

  good_story=0
  good_expression=0

  bad_story=0
  bad_expression=0

  for sent in sents:
    words=okt.morphs(sent)
    for word in words:
      if word in story:
        if sent not in story_sents:
          story_sents.append(sent)
    for word in words:
      if word in expression:
        if sent not in expression_sents:
          expression_sents.append(sent)

  if story_sents!=[]:

    Story_sents=tfd.transform(story_sents)
    for sent in Story_sents:
      if lr.predict(sent)==1:
        good_story+=1
      elif lr.predict(sent)==0:
        bad_story+=1

    story_score=((good_story)/(good_story+bad_story))*10
    story_scores.append(story_score)

  if expression_sents!=[]:      
    Expression_sents=tfd.transform(expression_sents)
    for sent in Expression_sents:
      if lr.predict(sent)==1:
        good_expression+=1
      elif lr.predict(sent)==0:
        bad_expression+=1

    expression_score=((good_expression)/(good_expression+bad_expression))*10 
    expression_scores.append(expression_score)
  
avg_story_score=round(sum(story_scores)/len(story_scores),2)
avg_expression_score=round(sum(expression_scores)/len(expression_scores),2)

In [8]:
##결과문
print(movie_title+"의 연출 점수는 "+str(avg_expression_score)+"입니다")
print(movie_title+"의 스토리 점수는 "+str(avg_story_score)+"입니다")

탑건: 매버릭의 연출 점수는 7.57입니다
탑건: 매버릭의 스토리 점수는 6.14입니다
